In [2]:
import openai
import json

import base64
import requests

import pandas as pd
import numpy as np

import uuid
import os

### Reading descriptions dataset

In [73]:

descrip_df = pd.DataFrame(columns=["Case_ID", "First_Party", "Second_Party", "Image_ID"])
descrip_df.head()

,Case_ID,First_Party,Second_Party,Image_ID


In [202]:
descrip_df = pd.read_csv("data\descriptions.csv")

In [191]:
descrip_df.head()

,Case_ID,First_Party,Second_Party,Image_ID
0,7c23480e-662e-4e8d-980e-b928d6fd0b90,كنت أقود باتجاه التقاطع وكان الضوء الأخضر في ص...,كنت أقود في الطريق ولم ألاحظ أن الإشارة كانت ح...,image.png
1,7c23480e-662e-4e8d-980e-b928d6fd0b90,كنت أقود عند التقاطع وكانت الإشارة خضراء لي، و...,كنت أقود وأتبع كل القواعد. وفجأة جاءت السيارة ...,image.png
2,7c23480e-662e-4e8d-980e-b928d6fd0b90,كنت أقود وفقًا للقانون والإشارة كانت خضراء بال...,كنت أقود ولم ألاحظ أن الإشارة كانت خضراء لي ول...,image.png
3,7c23480e-662e-4e8d-980e-b928d6fd0b90,كنت أقود باتجاه التقاطع وأعتقد أنني كنت من الم...,كنت أقود وألاحظ أن الإشارة كانت خضراء بالنسبة ...,image.png


### Starting connection with openai API

In [4]:
api_key = "sk-proj-UxsihIsXy5aq4Dq3G2DuT3BlbkFJwOAd6S4hvjcZN3nsCKvO"
openai.api_key = api_key

#### Get the descriptions by sending the image to chat-gpt 4o

In [5]:
#encoding function to encode the images before sending it to the API
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


In [83]:
#get the responses from the API and store them in the responses list, and save each image path in a list as well
images_folder = "data\imgs"
responses = []
images_paths = []


for image_name in os.listdir(images_folder):
  #image path
  image_path = os.path.join(images_folder, image_name)
  #store the image in the images_paths list
  images_paths.append(image_path)

  #encoded image
  base64_image = encode_image(image_path)

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4o",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            #prompt
            "text": """[no prose]
                      [Output Only JSON]
                      I have car crash image I want you to take place as the effected parities, where your will write a description of the first party side, then write the other party description of their perspective. 
                      and do it in 4 scenarios 
                      1st: both parties say the truth.
                      2nd: both parties laying
                      3rd: first party is laying.
                      4th: second party is laying
                      (all in arabic)
                      
                      so end result will be in json format:
                      {
                        scenario_n:{

                          first_party:

                          second_party:
                        },
                      }
                      """
          },
          {
            #include the image
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    #number of tokens
    "max_tokens": 650
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  print("---------------------I AM HERE---------------------")
  print(response.json())
  #store the response in responses list
  responses.append(response.json())
  
  print("---------------------I AM HERE2---------------------")

  # #get the descriptions content from the response
  # response_dict = response.json()
  # #extract the content
  # content = response_dict['choices'][0]['message']['content']

  # #clean the content string then turn it into json 
  # content = content.replace("```json", "").replace("```", "").strip()
  # # content = content.replace('",', '"')
  # descrip_json = json.loads(content)

  # #Store the descriptions in the dataframe
  # case_id = uuid.uuid4()

  # for i in range(1, 5):
  #     data = {
  #         "Case_ID": case_id,
  #         "First_Party": descrip_json[f'scenario_{i}']['first_party'],
  #         "Second_Party": descrip_json[f'scenario_{i}']['second_party'],
  #         "Image_ID": image_path
  #     }

  #     #create new row 
  #     new_row_df = pd.DataFrame([data])
  #     #add the new row to the descrip dataframe
  #     descrip_df = pd.concat([descrip_df, new_row_df], ignore_index=True)


  # descrip_df.head(20)

---------------------I AM HERE---------------------
{'id': 'chatcmpl-9Y7qxyjXGlUqrqGvZwRmGOj1DAUEl', 'object': 'chat.completion', 'created': 1717920367, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "scenario_1": {\n    "first_party": "كنت أسير بسرعتي القانونية وفجأة لاحظت أن السيارة أمامي توقفت فجأة. حاولت الفرملة ولكن لم أتمكن من إيقاف سيارتي في الوقت المناسب واصطدمت بها.",\n    "second_party": "كنت أقود بهدوء في المسار الأيمن عندما اضطررت لتخفيف السرعة بسبب ازدحام المرور أمامي، وفجأة شعرت بضربة قوية من الخلف عندما اصطدمت بي السيارة خلفي."\n  },\n  "scenario_2": {\n    "first_party": "كنت أقود ببطء في المسار الأيسر عندما ظهرت فجأة السيارة أمامي وتوقفت بدون سابق إنذار، ولم يكن أمامي وقت كافٍ للتوقف فحدث الاصطدام.",\n    "second_party": "بينما كنت أتباطأ عند إشارة المرور، جاءت السيارة الخلفية بسرعة عالية ولم تتمكن من التوقف في الوقت المناسب، فاصطدمت بي بقوة."\n  },\n  "scenario_3": {\n    "first_party": "كنت أقود ببطء 

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [84]:
#Store the responses and images id's in the dataframe
for r, img in zip(responses, images_paths):
    #get the descriptions content from the response
    response_dict = r
    #extract the content
    content = response_dict['choices'][0]['message']['content']

    #clean the content string then turn it into json 
    content = content.replace("```json",'').replace("```",'').strip()
    # content = content.replace('",', '"')
    
    descrip_json = json.loads(content)

    if isinstance(descrip_json, list):
        continue

    #Store the descriptions in the dataframe
    case_id = uuid.uuid4()

    for i in range(1, 5):
        data = {
            "Case_ID": case_id,
            "First_Party": descrip_json[f'scenario_{i}']['first_party'],
            "Second_Party": descrip_json[f'scenario_{i}']['second_party'],
            "Image_ID": img
        }

        #create new row 
        new_row_df = pd.DataFrame([data])
        #add the new row to the descrip dataframe
        descrip_df = pd.concat([descrip_df, new_row_df], ignore_index=True) 

In [45]:
print(type([]))

<class 'list'>


In [80]:
print(type(descrip_json))

<class 'dict'>


In [25]:
responses[9]

{'id': 'chatcmpl-9Y5yLJ2XzrLiZXyR6gumbmSO3jUlr',
 'object': 'chat.completion',
 'created': 1717913137,
 'model': 'gpt-4o-2024-05-13',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '```json\n{\n  "scenario_1": {\n    "first_party": "كنت أقود سيارتي ببطء في الازدحام عندما توقفت فجأة لأن السيارة أمامي توقفت، فجأة اصطدمت بي السيارة الأخرى من الخلف. أعتقد أن السائق خلفي لم يكن منتبهاً ولم يترك مسافة كافية بيننا.",\n    "second_party": "كنت أقود على الطريق بنفس سرعة السير، فجأة توقفت السيارة أمامي بشكل مفاجئ ولم أتمكن من التوقف في الوقت المناسب واصطدمت بها. لم أكن أتوقع أن تتوقف فجأة هكذا."\n  },\n  "scenario_2": {\n    "first_party": "كنت أقود ببطء في الطريق السريع ولم أتوقف فجأة. السيارة التي خلفي جاءت بسرعة مفرطة واصطدمت بي بالرغم من ترك مسافة كافية.",\n    "second_party": "كنت أقود بسرعة عادية وكنت منتبهاً للطريق. السيارة التي أمامي توقفت فجأة بدون سابق إنذار، ولم أتمكن من التوقف في الوقت المناسب بسبب سرعتها المفاجئة."\n  },\n  "scenario_3": {\n    "first_p

In [85]:
descrip_df.shape

(72, 4)

In [82]:
print(content)

Here is the JSON output you requested in Arabic, with the perspectives of both parties under four different scenarios:


{
  "scenario_1": {
    "first_party": "لقد كنت قادمًا من الجانب الأيمن عندما رأيت السيارة اللتي أمامي لا تتوقف عند الإشارة الحمراء. بعدما حاولت التوقف، لم أتمكن من تجنب الاصطدام.",
    "second_party": "كنت في طريقي عندما فاجأتني سيارة قادمة من الجانب الأيمن تجاوزت الإشارة الحمراء. لقد حاولت تفادي الاصطدام، لكن لم أتمكن من إيقاف السيارة في الوقت المناسب."
  },
  "scenario_2": {
    "first_party": "لقد كنت أقود بسرعة قانونية وكان لدي الإشارة الخضراء. فجأة، تجاوزت السيارة الثانية الإشارة الحمراء وضربتني.",
    "second_party": "كنت أقود ببطء شديد على الرغم من أن الإشارة كانت حمراء، السيارة الأخرى كانت مسرعة جدا لم أتمكن من تفاديها."
  },
  "scenario_3": {
    "first_party": "كنت أقود بحذر شديد وكان لدي الإشارة الخضراء، وفجأة جاءت السيارة الأخرى وتجاوزت الإشارة الحمراء وصدمتني.",
    "second_party": "بصراحة، كانت الإشارة خضراء للطرف الآخر، وكنت أنا من تجاوزت الإشارة الحم

In [86]:
descrip_df.head(40)

,Case_ID,First_Party,Second_Party,Image_ID
0,7fc3f4b9-f9d1-4da4-bdc2-7801853c0a91,كنت أقود سيارتي البيضاء على الطريق السريع وفجأ...,كنت أقود سيارتي الفضية على الطريق السريع وعندم...,data\imgs\1.jpeg
1,7fc3f4b9-f9d1-4da4-bdc2-7801853c0a91,السيارة الأمامية توقفت فجأة بدون أي سبب مبرر، ...,السيارة الخلفية كانت تسرع بشكل غير طبيعي ولم ت...,data\imgs\1.jpeg
2,7fc3f4b9-f9d1-4da4-bdc2-7801853c0a91,كنت أسير بسرعة طبيعية والسيارة الأمامية توقفت ...,كنت أسير بسرعة طبيعية وتوقفت فجأة لتفادي حركة ...,data\imgs\1.jpeg
3,7fc3f4b9-f9d1-4da4-bdc2-7801853c0a91,كنت أسير بسرعة طبيعية وحافظت على المسافة الأما...,السيارة الخلفية كانت تسرع بشكل غير طبيعي ولم ت...,data\imgs\1.jpeg
4,113bd662-3b93-41e1-a4e6-95fad5ca4955,كنت أقود سيارتي السوداء وكنت على حق الطريق. فج...,كنت أقود سيارتي الفضية وكنت على حق الطريق. ولك...,data\imgs\10.jpeg
5,113bd662-3b93-41e1-a4e6-95fad5ca4955,كنت أقود سيارتي السوداء على السرعة المحددة وفي...,كنت أقود سيارتي الفضية بحذر وداخل السرعة المحد...,data\imgs\10.jpeg
6,113bd662-3b93-41e1-a4e6-95fad5ca4955,كنت أقود سيارتي السوداء بأمان وداخل السرعة الم...,كانت السيارة السوداء تسير بسرعة كبيرة وغير متو...,data\imgs\10.jpeg
7,113bd662-3b93-41e1-a4e6-95fad5ca4955,كنت أقود سيارتي السوداء على السرعة المحددة وفي...,كنت أقود سيارتي الفضية بحذر وداخل السرعة المحد...,data\imgs\10.jpeg
8,76945d13-fb53-4953-9622-223e512a853a,كنت أسير بسرعتي القانونية وفجأة لاحظت أن السيا...,كنت أقود بهدوء في المسار الأيمن عندما اضطررت ل...,data\imgs\1.jpeg
9,76945d13-fb53-4953-9622-223e512a853a,كنت أقود ببطء في المسار الأيسر عندما ظهرت فجأة...,بينما كنت أتباطأ عند إشارة المرور، جاءت السيار...,data\imgs\1.jpeg


In [87]:
descrip_df.to_csv("data\descriptions.csv", index=False)

### Get the fault assessment based on the generated descriptions using chat-gpt 4o model

In [51]:
sttr = """هذه النتيجة ستكون الوصف كنص JSON و سيكون لكل سيناريو:

```json
{
  "scenario_1": {
    "first_party": "كنت أسير بالاتجاه اليمين كما هو مطلوب، وفجأة قام السائق الأخر بقطع الإشارة وتوجه نحوي مما تسبب في الاصطدام.",
    "second_party": "كنت أقترب من التقاطع بشكل طبيعي، وعندما بدأت بالمرور عند النقطة كانت السيارات قد توقفت، وأثناء مروري فوجئت بالسيارة الأخرى تصطدم بي من الجهة اليمنى."
  },
  "scenario_2": {
    "first_party": "كنت أسير ببراعة وبالسرعة القانونية، لم أر أي سيارة أخرى في التقاطع وفجأت جاءت السيارة الأخرى مسرعة واصطدمت بي.",
    "second_party": "كنت أسير بالسرعة القانونية والتزمت بالمسار الخاص بي، فاقتربت السيارة الأخرى من جانبي واصطدمت بي بعنف."
  },
  "scenario_3": {
    "first_party": "كنت أسير وفق القواعد والسرعة المناسبة، والسيارة الأخرى قطعت الإشارة بشكل مفاجئ مما تسبب بالاصطدام.",
    "second_party": "كنت أقترب من الإشارة بشكل طبيعي، وفجأة جاءت السيارة المقابلة واصطدمت بي دون أي محاولة لتخفيف السرعة."
  },
  "scenario_4": {
    "first_party": "كنت أسير بزيادة في السرعة ولكن بقيت في مساري، والسيارة الأخرى جاءت من الجهة المعاكسة واصطدمت بي فجأة.",
    "second_party": "كنت متوقفاً عند الإشارة، وقمت بعبورها عندما تحولت إلى الخضراء، ولكن فجأة جاءت السيارة الأخرى بشكل سريع واصطدمت بي."
  }
}
"""

In [52]:
sttr = sttr.split("```json")

In [56]:
print(sttr[1])


{
  "scenario_1": {
    "first_party": "كنت أسير بالاتجاه اليمين كما هو مطلوب، وفجأة قام السائق الأخر بقطع الإشارة وتوجه نحوي مما تسبب في الاصطدام.",
    "second_party": "كنت أقترب من التقاطع بشكل طبيعي، وعندما بدأت بالمرور عند النقطة كانت السيارات قد توقفت، وأثناء مروري فوجئت بالسيارة الأخرى تصطدم بي من الجهة اليمنى."
  },
  "scenario_2": {
    "first_party": "كنت أسير ببراعة وبالسرعة القانونية، لم أر أي سيارة أخرى في التقاطع وفجأت جاءت السيارة الأخرى مسرعة واصطدمت بي.",
    "second_party": "كنت أسير بالسرعة القانونية والتزمت بالمسار الخاص بي، فاقتربت السيارة الأخرى من جانبي واصطدمت بي بعنف."
  },
  "scenario_3": {
    "first_party": "كنت أسير وفق القواعد والسرعة المناسبة، والسيارة الأخرى قطعت الإشارة بشكل مفاجئ مما تسبب بالاصطدام.",
    "second_party": "كنت أقترب من الإشارة بشكل طبيعي، وفجأة جاءت السيارة المقابلة واصطدمت بي دون أي محاولة لتخفيف السرعة."
  },
  "scenario_4": {
    "first_party": "كنت أسير بزيادة في السرعة ولكن بقيت في مساري، والسيارة الأخرى جاءت من الجهة المعاكسة واص